In [ ]:
pip install pandas statsmodels
pip install yfinance statsmodels

In [ ]:
import datetime
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# Data fetched from FRED(Federal Reserve Economic Data) and BLS(Bureau of Labor Statistics)
macro_data = sm.datasets.macrodata.load_pandas().data

# Fetch S&P 500 data from Yahoo Finance using yfinance
start_date = '1959-01-01'
end_date = '2009-12-31'
sp500_data = yf.download('^GSPC', start=start_date, end=end_date)

# Resample S&P 500 data to quarterly frequency to match macro_data
sp500_data_quarterly = sp500_data['Adj Close'].resample('Q').last()

# Create a date range that matches the macroeconomic data's index
date_range = pd.date_range(start='1959-01-01', periods=len(macro_data), freq='Q')

# Ensure both data series are aligned by reindexing
sp500_data_quarterly = sp500_data_quarterly.reindex(date_range)

# Combine macroeconomic data with S&P 500 data
macro_data.index = date_range
combined_data = macro_data.copy()
combined_data['sp500'] = sp500_data_quarterly.values

# Display the combined data
combined_data.head()

[*********************100%%**********************]  1 of 1 completed


,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint,sp500
1959-03-31,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00,55.439999
1959-06-30,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74,58.470001
1959-09-30,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09,56.880001
1959-12-31,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06,59.889999
1960-03-31,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19,55.340000


In [ ]:
# Extract relevant columns
df = combined_data[['year', 'quarter', 'realgdp', 'realcons', 'realinv', 'cpi', 'unemp', 'pop', 'infl', 'tbilrate','sp500']]

# Convert 'year' and 'quarter' to integers
df['year'] = df['year'].astype(int)
df['quarter'] = df['quarter'].astype(int)

# Convert year and quarter to datetime format
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['quarter'].astype(str), format='%Y-%m')

# Set date as index
df.set_index('date', inplace=True)

# Let's define recession as 1 if real GDP growth for a quarter is negative
df['Recession'] = (df['realgdp'].diff() < 0).astype(int)

print(df.head())

            year  quarter   realgdp  realcons  realinv    cpi  unemp      pop  \
date                                                                            
1959-01-01  1959        1  2710.349    1707.4  286.898  28.98    5.8  177.146   
1959-02-01  1959        2  2778.801    1733.7  310.859  29.15    5.1  177.830   
1959-03-01  1959        3  2775.488    1751.8  289.226  29.35    5.3  178.657   
1959-04-01  1959        4  2785.204    1753.7  299.356  29.37    5.6  179.386   
1960-01-01  1960        1  2847.699    1770.5  331.722  29.54    5.2  180.007   

            infl  tbilrate      sp500  Recession  
date                                              
1959-01-01  0.00      2.82  55.439999          0  
1959-02-01  2.34      3.08  58.470001          0  
1959-03-01  2.74      3.82  56.880001          1  
1959-04-01  0.27      4.33  59.889999          0  
1960-01-01  2.31      3.50  55.340000          0  


<ipython-input-50-5cb8ca1dab26>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].astype(int)
<ipython-input-50-5cb8ca1dab26>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'] = df['quarter'].astype(int)
<ipython-input-50-5cb8ca1dab26>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [ ]:
# Define features and target
X = df.drop(['Recession', 'year', 'quarter'], axis=1) # Drop unnecessary columns
y = df['Recession']

In [ ]:
X.shape

(203, 9)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}')

Accuracy: 92.68
